Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

In [431]:
from azureml.core import Workspace, Experiment, Run, Environment
from azureml.pipeline.core import Pipeline, PipelineData
from azureml.data.data_reference import DataReference
from azureml.pipeline.steps import PythonScriptStep
from azureml.core.runconfig import RunConfiguration
from azureml.core.compute import AmlCompute,ComputeTarget
from azureml.core.datastore import Datastore



In [432]:
ws = Workspace.from_config()
default_datastore = ws.get_default_datastore()
account_name = default_datastore.account_name
account_key = default_datastore.account_key
#TODO: load variables from config file

exp_name = ""
compute_name = "testcompute"
env_name = ""

project_dir = '../src'
script_name = 'batch_score.py'
model_name = ""
# model_file_name = 'my_model.pkl'

prediction_dataset_name = ""
result_output_dir = 'outputs/'
result_dataset = ""
datastore_output_name = ""
pipeline_name = ""

container_name = ""


In [433]:
batchscore_input = default_datastore.as_mount()


batchscore_ds = Datastore.register_azure_blob_container(ws, 
                    datastore_name=datastore_output_name, 
                    container_name=container_name, 
                    account_name=account_name,
                    account_key = account_key,
                    create_if_not_exists=True)

batchscore_output = DataReference(
    batchscore_ds, 
    data_reference_name='my_output_dir', 
    path_on_datastore=result_output_dir, 
    mode='mount', 
    overwrite=True
    )



In [434]:
compute_target = ComputeTarget(workspace=ws, name=compute_name)

In [435]:
env = Environment(name='Inference env')
for pip_package in ["joblib","scikit-learn","pandas","azureml-sdk"]:
    env.python.conda_dependencies.add_pip_package(pip_package)

In [436]:
run_config = RunConfiguration()
run_config.environment = env


In [437]:

scoring_step = PythonScriptStep(
    name = "Batch Scoring",
    script_name = script_name,
    compute_target = compute_target,
    source_directory = project_dir,
    inputs=[batchscore_output]
    arguments = [
        '--input-dir',
        batchscore_input,
        '--model-name',
        model_name,
        '--dataset-name',
        prediction-dataset_name,
        '--forecast-name',
        result_dataset
    ],
    runconfig = run_config,
    allow_reuse = True
)



In [438]:
pipeline = Pipeline(workspace=ws, steps=[scoring_step])
pipeline.validate()

Step Batch Scoring is ready to be created [ddd89787]


[]

In [439]:
pipeline_run = Experiment(ws,exp_name).submit(pipeline)

Created step Batch Scoring [ddd89787][fc1d6289-267e-4993-8d0c-382cb754a0b0], (This step will run and generate new outputs)
Submitted PipelineRun 8a0bd8cb-471d-4a2c-936d-7feb2e131b78
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/batch-scoring/runs/8a0bd8cb-471d-4a2c-936d-7feb2e131b78?wsid=/subscriptions/ed835daf-8df9-4922-a58f-74a80da0b5c0/resourcegroups/mlops-uc-sales/workspaces/mlouc1daml


In [440]:
pipeline_run.get_status()

'Running'

In [441]:
if pipeline_run.get_status() == 'Completed':
    published_endpoint, published_pipeline = pipeline_run.publish_pipeline(
        name=pipeline_name, description="Batch scoring")
